## **Three dims reconstruction of 3D point cloud**

    This notebook demonstrate:
    1.3D point cloud reconstruction of the whole body;
    2.3D point cloud reconstruction of the tissues;
    3.Visualize the 3D distribution of gene expression;
    4.3D Coordinate Outlier Detection and Removal;
    5.Voxelize the point cloud;
    6.Orthogonal slicing of 3D point cloud data;
    7.Axial slicing of 3D point cloud data.

Although most of the functions of 3D reconstruction can be used in jupyter notebook, I prefer to use an edition like PyCharm.
Most widgets cannot be used in jupyter notebook, but only in vtk rendering window, so PyCharm is highly recommended.

### **Packages**

In [1]:
import anndata as ad
import stDrosophila as sd

/home/yao/anaconda3/envs/stDrosophila-release/lib/python3.7/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### **Example data (Drosophila E16-18-a)**

In [2]:
file = r"/home/yao/BGIpy37_pytorch113/drosophila_E16_18/E16-18_a_SCT_anno.h5ad"
adata = ad.read(file)
adata.obs["x"] = adata.obs["x"] - adata.obs["x"].min()
adata.obs["y"] = adata.obs["y"] - adata.obs["y"].min()
adata.obs["z"] = adata.obs["z"] - 4.9
adata.obs[["x", "y", "z"]] = adata.obs[["x", "y", "z"]].round(2)
adata.obsm["spatial"] = adata.obs[["x", "y", "z"]].values
print(adata)
print(adata.obs["anno"].unique())

AnnData object with n_obs × n_vars = 14634 × 518
    obs: 'slice', 'x', 'y', 'z', 'anno'
    obsm: 'raw_spatial', 'spatial'
    layers: 'raw_counts'
['salivary gland', 'epidermis', 'CNS', 'carcass', 'fat body', 'muscle', 'trachea', 'midgut', 'hemolymph', 'foregut']
Categories (10, object): ['CNS', 'carcass', 'epidermis', 'fat body', ..., 'midgut', 'muscle', 'salivary gland', 'trachea']


### **3D reconstrcution of the whole body**

3D point cloud can be reconstructed by this method.

#### `sd.tl.construct_pcd`: Construct a point cloud based on 3D coordinate information.
- `coordsby`: The key that stores 3D coordinate information in adata.obsm.
- `groupby`: The key that stores clustering or annotation information in adata.obs, or a gene name in adata.var.
- `key_added`: Store `groupby` information in pcd.cell_data[key_added]
#### `sd.pl.three_d_plot`: Visualize reconstructed 3D meshes.
- `key`: The key of pcd.cell_data which stores `groupby` information.
- `jupyter`: Whether to plot in jupyter notebook.

In [3]:
pcd = sd.tl.construct_pcd(adata=adata, coordsby="spatial", groupby="anno", key_added="groups", colormap="rainbow", alphamap=1.0)
sd.pl.three_d_plot(mesh=pcd, key="groups", jupyter=True, style="points", point_size=10)

### **3D reconstrcution of the tissue**

If you only want to display one of the tissues, you can use the following two methods (here takes the visualization of the fat body as an example):

1.Use the `mask` parameter to set the tissues that do not need to be displayed as mask.

In [4]:
mask = ['salivary gland', 'epidermis', 'CNS', 'carcass', 'muscle', 'trachea', 'midgut', 'hemolymph', 'foregut']
fb_pcd = sd.tl.construct_pcd(adata=adata, coordsby="spatial", groupby="anno", key_added="groups", mask=mask, colormap="skyblue", alphamap=1.0)
sd.pl.three_d_plot(mesh=fb_pcd, key="groups", jupyter=True)

2.Modify the original adata so that only one tissue remains in the `groupby`.

In [5]:
fb_adata = adata[adata.obs["anno"] == "fat body", :].copy()
fb_pcd = sd.tl.construct_pcd(adata=fb_adata, coordsby="spatial", groupby="anno", key_added="groups", colormap="skyblue", alphamap=1.0)
sd.pl.three_d_plot(mesh=fb_pcd, key="groups", jupyter=True)

### **3D distribution of gene expression**
Sometimes you want to display the expression values of certain genes on the 3D reconstructed point cloud model,
you can set the `groupby` to the gene name you want to display, and make sure that the gene name exists in adata.var.

In this example I show the gene expression distribution of the Adh gene in the reconstructed fat body 3D point cloud.

In [6]:
fb_adata = adata[adata.obs["anno"] == "fat body", :].copy()
fb_pcd = sd.tl.construct_pcd(adata=fb_adata, coordsby="spatial", groupby="Adh", key_added="groups", colormap="hot_r", alphamap=1.0)
sd.pl.three_d_plot(mesh=fb_pcd, key="groups", jupyter=True)

### **3D Coordinate Outlier Detection and Removal**
In the above examples, it can be found that the three-dimensional coordinate distribution of the fat body is relatively scattered,
which may be due to the unsatisfactory effect of clustering and annotation, resulting in the existence of outliers in three-dimensional coordinates.

Here we provide the following two methods for removing outliers：

1.Outlier detection based on kernel density estimation and EllipticEnvelope algorithm.

In [7]:
fb_adata = adata[adata.obs["anno"] == "fat body", :].copy()
fb_adata = sd.tl.om_EllipticEnvelope(adata=fb_adata, coordsby="spatial", threshold=0.2)
fb_adata = sd.tl.om_kde(adata=fb_adata, coordsby="spatial", threshold=0.2, kernel="gaussian", bandwidth=1.0)
fb_pcd = sd.tl.construct_pcd(adata=fb_adata, coordsby="spatial", groupby="Adh", key_added="groups", colormap="hot_r", alphamap=1.0)
sd.pl.three_d_plot(mesh=fb_pcd, key="groups", jupyter=True)

2.Manually pick parts that are artificially considered outliers in the interactive interface.

Since it is difficult to use widgets in jupyter notebook, please do this method in an edition similar to PyCharm.

In [8]:
fb_adata = adata[adata.obs["anno"] == "fat body", :].copy()
fb_pcd = sd.tl.construct_pcd(adata=fb_adata, coordsby="spatial", groupby="Adh", key_added="groups", colormap="hot_r", alphamap=1.0)
fb_pcd = sd.tl.three_d_pick(mesh=fb_pcd,key="groups",pick_method="rectangle", invert=True, merge=True)

/home/yao/stDrosophila-release/stDrosophila/tools/three_d_reconstruction/picking.py:40: UserWarning: The mesh should be a pyvista.UnstructuredGrid object.
  warnings.warn("The mesh should be a pyvista.UnstructuredGrid object.")


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

IndexError: list index out of range

### **Voxelize the point cloud**
Voxelized point cloud. For point cloud data, slicing can only be done after voxelization.

In [9]:
pcd = sd.tl.construct_pcd(adata=adata, coordsby="spatial", groupby="anno", key_added="groups", colormap="rainbow", alphamap=1.0)
v_pcd = sd.tl.voxelize_pcd(pcd=pcd, voxel_size=[0.5, 0.5, 0.7])
sd.pl.three_d_plot(mesh=v_pcd, key="groups", jupyter=True)

### **Orthogonal slicing**

Create three orthogonal slices through the dataset on the three cartesian planes.
Here we provide the following two methods of orthogonal slicing:

1.Slice by entering the `center` to customize the center position of the orthogonal slice.
- `center`: A 3-length sequence specifying the position which slices are taken. Defaults to the center of the mesh.

In [11]:
o_slices = sd.tl.three_d_slice(mesh=v_pcd, key="groups", slice_method="orthogonal", slice_method_args={"center": None}, interactive=False)
# What is generated by orthogonal slicing is a list containing three slices, we need to combine the three slices and output at the same time.
merge_o_slice = sd.tl.collect_mesh(o_slices)
sd.pl.three_d_plot(mesh=merge_o_slice, key="groups", jupyter=True)

2.Orthogonal slicing through the interactive interface.
If you find the center parameter difficult to locate, you can choose to manually adjust the center of the orthogonal slice through the interactive interface.

Since it is difficult to use widgets in jupyter notebook, please do this method in an edition similar to PyCharm.

In [ ]:
o_slices = sd.tl.three_d_slice(mesh=v_pcd, key="groups", slice_method="orthogonal", interactive=True)
# What is generated by orthogonal slicing is a list containing three slices, we need to combine the three slices and output at the same time.
merge_o_slice = sd.tl.collect_mesh(o_slices)
sd.pl.three_d_plot(mesh=merge_o_slice, key="groups", jupyter=True)

### **Axial slicing**

Here we provide the following two methods of orthogonal slicing:

1.Slice by entering the following three parameters to customize the axial slice
- `n_slices`: The number of slices to create along a specified axis. Only works when `interactive` is False.
- `axis`: The axis to generate the slices along.
- `center`: A 3-length sequence specifying the position which slices are taken. Defaults to the center of the mesh.

In [12]:
a_slices = sd.tl.three_d_slice(mesh=v_pcd, key="groups", slice_method="axis", slice_method_args={"n_slices": 10, "axis": "x", "center": None}, interactive=False)
# What is generated by orthogonal slicing is a list containing three slices, we need to combine the three slices and output at the same time.
merge_a_slice = sd.tl.collect_mesh(a_slices)
sd.pl.three_d_plot(mesh=merge_a_slice, key="groups", jupyter=True)

2.Axial slicing through the interactive interface.
If you think entering parameters for slicing is too difficult for you, you can choose to manually select the required slice through the interactive interface.
But this method can only create one slice at a time.

Since it is difficult to use widgets in jupyter notebook, please do this method in an edition similar to PyCharm.

In [ ]:
a_slices = sd.tl.three_d_slice(mesh=v_pcd, key="groups", slice_method="axis", interactive=True)
sd.pl.three_d_plot(mesh=a_slices[0], key="groups", jupyter=True)